<a href="https://colab.research.google.com/github/Evertonferrg/imersaoAluraia_projeto_analise_credito/blob/main/imersaoalura_sistema_analise_credito.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip -q install google-genai

In [ ]:
!pip install google-generativeai pandas altair

In [ ]:
!pip install google-adk


In [ ]:
!pip install pandas altair

In [ ]:
!pip install google-generativeai

In [ ]:
import os
from google.colab import userdata
from google import genai
from IPython.display import HTML, Markdown
from google.adk.agents import Agent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools import google_search
from google.genai import types
from datetime import date
import textwrap
import warnings
import pandas as pd  # Para criar o gráfico de riscos
import altair as alt # Para criar o gráfico de riscos

warnings.filterwarnings("ignore")



In [ ]:
import os
from google.colab import userdata

os.environ["GOOGLE_API_KEY"] = userdata.get('GOOGLE_API_KEY')


In [ ]:
# configura o cliente da SDK do Gemini

from google import genai

client = genai.Client()

MODEL_ID = "gemini-2.0-flash"

In [ ]:
# Funcao auxiliar para enviar mensagem ao agente
def call_agent(agent: Agent, message_text: str) -> str:
    session_service = InMemorySessionService()
    session = session_service.create_session(app_name=agent.name, user_id="user1", session_id="session1")
    runner = Runner(agent=agent, app_name=agent.name, session_service=session_service)
    content = types.Content(role="user", parts=[types.Part(text=message_text)])

    final_response = ""
    for event in runner.run(user_id="user1", session_id="session1", new_message=content):
        if event.is_final_response():
            for part in event.content.parts:
                if part.text is not None:
                    final_response += part.text
                    final_response += "\n"
    return final_response

In [ ]:
# Funcção auxiliar para exibir texto formatado
def to_markdown(text):
    text = text.replace('•', '  *')
    return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))


def buscar_info_empresa(nome_ou_cnpj):
    """
    Agente para buscar informações públicas da empresa usando o Google.
    """
    agente_busca_empresa = Agent(
        name="agente_busca_empresa",
        model=MODEL_ID, # Correção: Use MODEL_ID em vez de MODEL
        instruction="""
            Você é um especialista em análise de crédito e deve buscar informações
            públicas e relevantes sobre a empresa fornecida. A busca deve ser feita
            usando o nome ou CNPJ da empresa. Foque em informações como:

            -   Situação cadastral (Ativa, Inativa, etc.)
            -   Data de abertura
            -   Endereço
            -   Telefone
            -   E-mail
            -   Atividade econômica principal e secundárias
            -   Capital social
            -   Quadro de sócios e administradores (QSA)
            -   Faturamento dos últimos 3 anos (se disponível publicamente)
            -   Endividamento (se disponível publicamente)
            -   Processos judiciais e dívidas (se disponíveis publicamente)
            -   Notícias recentes e relevantes sobre a empresa
            -   Avaliação de clientes (se disponível publicamente)

            Apresente as informações de forma clara e organizada.
            Se não encontrar uma informação, mencione que não foi encontrada.
        """,
       description="Agente que busca informações sobre empresas",
        tools=[google_search]
    )
    entrada_busca = f"Nome ou CNPJ da empresa: {nome_ou_cnpj}"
    return call_agent(agente_busca_empresa, entrada_busca)

In [ ]:
def coletar_informacoes_manuais():
    """
    Função para coletar informações da empresa diretamente do usuário.
    """
    informacoes = {}
    informacoes['nome_empresa'] = input("Nome da Empresa: ")
    informacoes['cnpj'] = input("CNPJ: ")
    informacoes['situacao_cadastral'] = input("Situação Cadastral: ")
    informacoes['data_abertura'] = input("Data de Abertura: ")
    informacoes['endereco'] = input("Endereço: ")
    informacoes['telefone'] = input("Telefone: ")
    informacoes['email'] = input("E-mail: ")
    informacoes['atividade_principal'] = input("Atividade Econômica Principal: ")
    informacoes['atividades_secundarias'] = input("Atividades Econômicas Secundárias: ")
    informacoes['capital_social'] = input("Capital Social: ")
    informacoes['qsa'] = input("Quadro de Sócios e Administradores (QSA): ")
    informacoes['faturamento_ultimos_3_anos'] = input("Faturamento dos Últimos 3 Anos: ")
    informacoes['endividamento'] = input("Endividamento: ")
    informacoes['processos_judiciais_dividas'] = input("Processos Judiciais e Dívidas: ")
    informacoes['noticias_relevantes'] = input("Notícias Recentes e Relevantes: ")
    informacoes['avaliacao_clientes'] = input("Avaliação de Clientes: ")
    return informacoes


In [ ]:
def avaliar_risco_credito(informacoes):
    """
    Agente para avaliar o risco de crédito da empresa.
    """
    agente_avaliacao_risco = Agent(
        name="agente_avaliacao_risco",
        model=MODEL_ID,
        instruction="""
            Você é um especialista em análise de crédito e deve avaliar o risco de crédito
            de uma empresa com base nas informações fornecidas. Considere os seguintes fatores:

            -   **Situação Cadastral:** Empresas com situação irregular (Inativa, Suspensa, etc.)
                representam maior risco.
            -   **Tempo de Atividade:** Empresas mais novas podem ter histórico financeiro limitado,
                aumentando o risco.
            -   **Endividamento:** Níveis elevados de endividamento indicam maior dificuldade em
                honrar compromissos financeiros.
            -   **Faturamento:** O faturamento deve ser suficiente para cobrir as despesas e gerar lucro.
            -   **Processos Judiciais e Dívidas:** A existência de processos e dívidas pode comprometer
                a capacidade de pagamento da empresa.
            -   **Setor de Atuação:** Alguns setores são mais voláteis e podem apresentar maior risco.
            -   **Avaliação de Clientes:** Avaliações negativas podem indicar problemas na gestão ou
                na qualidade dos produtos/serviços.
            -   **Capital Social:** Um capital social robusto pode indicar maior capacidade da empresa
                de arcar com suas obrigações.
            -   **QSA:** A experiência e reputação dos sócios e administradores são importantes.
            -   **Notícias Recentes:** Notícias negativas podem afetar a imagem e a saúde financeira da empresa.

            Atribua uma nota de risco de 1 a 5, onde 1 é risco muito baixo e 5 é risco muito alto.
            Apresente a nota de risco em uma linha e a justificativa em outra.
        """,
        description="Agente que avalia o risco de crédito de uma empresa",
    )
    entrada_avaliacao = f"Informações da Empresa: {informacoes}"
    return call_agent(agente_avaliacao_risco, entrada_avaliacao)

In [ ]:
def gerar_relatorio_credito(nome_empresa, avaliacao_risco, justificativa_risco, informacoes):
    """
    Função para gerar um relatório de crédito completo.
    """

    relatorio = f"""
    ## Relatório de Análise de Crédito

    **Nome da Empresa:** {nome_empresa}

    ### Avaliação de Risco de Crédito
    -   **Nota de Risco:** {avaliacao_risco} (1 - Risco Muito Baixo, 5 - Risco Muito Alto)
    -   **Justificativa:** {justificativa_risco}

    ### Decisão sobre Aumento de Crédito
    Com base na análise das informações disponíveis, a recomendação é:
    """
    if avaliacao_risco <= 2:
        relatorio += "**Conceder o aumento de crédito.** A empresa apresenta baixo risco e boa capacidade de pagamento."
    elif avaliacao_risco <= 3:
        relatorio += "**Avaliar com cautela o aumento de crédito.** A empresa apresenta risco moderado. Recomenda-se uma análise mais aprofundada."
    else:
        relatorio += "**Não conceder o aumento de crédito.** A empresa apresenta alto risco de inadimplência."

    relatorio += """

    ### Informações Detalhadas da Empresa:
    """
    for chave, valor in informacoes.items():
        relatorio += f"-   **{chave.capitalize()}:** {valor}\n"

    return relatorio


In [ ]:
def gerar_grafico_risco(avaliacao_risco):
    """
    Função para gerar um gráfico de risco usando Altair.
    """
    # Define os dados para o gráfico
    data = pd.DataFrame({'Risco': ['Risco'], 'Nota': [avaliacao_risco]})

    # Define a escala de cores
    color_scale = alt.Scale(domain=[1, 2, 3, 4, 5],
                            range=['green', 'blue', 'yellow', 'orange', 'red'])

    # Cria o gráfico de barras
    chart = alt.Chart(data).mark_bar().encode(
        x=alt.X('Risco', axis=None), # Remove o rótulo do eixo x
        y=alt.Y('Nota', title='Nota de Risco'),
        color=alt.Color('Nota', scale=color_scale),
        tooltip=['Nota']
    ).properties(
        title='Gráfico de Risco de Crédito'
    )

    # Adiciona rótulos ao gráfico
    text = chart.mark_text(
        align='center',
        baseline='middle',
        dy=-5  # Ajusta a posição vertical do texto
    ).encode(
        text='Nota'
    )

    # Combina o gráfico de barras com os rótulos
    final_chart = chart + text

    # Salva o gráfico em um arquivo JSON
    final_chart.save('grafico_risco_credito.json')
    return final_chart
    #return HTML(final_chart.to_html()) # Exibe o gráfico no Colab

In [ ]:
def main():
    """
    Função principal para executar o sistema de análise de crédito.
    """
    data_de_hoje = date.today().strftime("%d/%m/%Y")
    print("🚀 Iniciando o Sistema de Análise de Crédito 🚀")

    # --- Obter informações da empresa ---
    opcao = input("❓ Deseja buscar informações da empresa pelo nome/CNPJ (1) ou fornecer as informações manualmente (2)? Digite 1 ou 2: ")
    if opcao == "1":
        nome_ou_cnpj = input("Digite o Nome ou CNPJ da empresa: ")
        info_empresa = buscar_info_empresa(nome_ou_cnpj)
        print("\n--- 📝 Resultado da Busca de Informações da Empresa ---\n")
        display(to_markdown(info_empresa))
        print("--------------------------------------------------------------")
        # Converter a string para dicionário
        info_empresa_dict = {}
        for linha in info_empresa.split('\n'):
            if ":" in linha:
                chave, valor = linha.split(":", 1)
                info_empresa_dict[chave.strip().lower().replace(" ", "_")] = valor.strip()
    elif opcao == "2":
        info_empresa_dict = coletar_informacoes_manuais()
    else:
        print("Opção inválida. Encerrando o programa.")
        return

      # --- Avaliar o risco de crédito ---
    avaliacao_risco_str = avaliar_risco_credito(info_empresa_dict)
    partes = avaliacao_risco_str.split('\n')
    avaliacao_risco = partes[0].split(': ')[1]
    justificativa_risco = partes[1].split(': ')[1]
    avaliacao_risco = int(avaliacao_risco) # Converter avaliacao_risco para int
    print("\n--- 📝 Resultado da Avaliação de Risco de Crédito ---\n")
    print(f"Nota de Risco: {avaliacao_risco}")
    print(f"Justificativa: {justificativa_risco}")
    print("--------------------------------------------------------------")

    # --- Gerar o relatório de crédito ---
    nome_empresa = info_empresa_dict.get('nome_empresa', 'Nome da Empresa Não Informado')
    relatorio_credito = gerar_relatorio_credito(nome_empresa, avaliacao_risco, justificativa_risco, info_empresa_dict)
    print("\n--- 📝 Relatório de Crédito ---\n")
    display(to_markdown(relatorio_credito))
    print("--------------------------------------------------------------")

    # --- Gerar o gráfico de risco ---
    grafico_risco = gerar_grafico_risco(avaliacao_risco)
    print("\n--- 📊 Gráfico de Risco de Crédito ---\n")
    display(grafico_risco) # Exibe o gráfico no Colab
    print("--------------------------------------------------------------")

if __name__ == "__main__":
    main()


🚀 Iniciando o Sistema de Análise de Crédito 🚀
❓ Deseja buscar informações da empresa pelo nome/CNPJ (1) ou fornecer as informações manualmente (2)? Digite 1 ou 2: 1
Digite o Nome ou CNPJ da empresa: Google

--- 📝 Resultado da Busca de Informações da Empresa ---



> Com base nas informações disponíveis, aqui está um resumo sobre a empresa Google:
> 
> *   **Situação Cadastral:** Não foi encontrada informação direta sobre a situação cadastral da Google.
> *   **Data de Abertura:** A Google foi fundada como empresa privada em 4 de setembro de 1998. O nome de domínio "Google" foi registrado em 15 de setembro de 1997.
> *   **Endereço:** A sede da Google é no Googleplex, em Mountain View, Califórnia, Estados Unidos. A Google possui escritórios em cerca de 60 países.
> *   **Telefone:** Não foi encontrado um número de telefone geral para contato com a Google.
> *   **E-mail:** Não foi encontrado um e-mail geral para contato com a Google.
> *   **Atividade Econômica Principal e Secundárias:** A Google atua nas áreas de internet, computação em nuvem, software, hardware, inteligência artificial e publicidade. A empresa lucra principalmente através da publicidade pelo AdWords.
> *   **Capital Social:** Essa informação não foi encontrada.
> *   **Quadro de Sócios e Administradores (QSA):** Os fundadores são Larry Page e Sergey Brin. O CEO atual do Google e da Alphabet é Sundar Pichai. Anat Ashkenazi é a CFO e Thomas Kurian é o CEO da Google Cloud.
> *   **Faturamento dos Últimos 3 Anos:**
> 
>     *   2021: A Alphabet (dona do Google) teve uma receita de US$ 257,637 bilhões e um lucro líquido de US$ 76,033 bilhões.
>     *   2022: A receita do Google no terceiro trimestre foi de US$ 69,1 bilhões.
>     *   2020: A empresa faturou 125,6 bilhões de dólares nos três primeiros trimestres.
> *   **Endividamento:** Não foram encontradas informações sobre o endividamento da Google.
> *   **Processos Judiciais e Dívidas:** Em maio de 2025, a justiça dos EUA determinou que o Google divida seu negócio de publicidade.
> *   **Notícias Recentes e Relevantes sobre a Empresa:**
> 
>     *   Maio de 2025: Hackers que atacaram varejistas do Reino Unido agora estão mirando nos EUA.
>     *   Maio de 2025: O Google anunciou um novo visual para o Android 16.
>     *   Maio de 2025: A justiça dos EUA determinou que o Google divida seu negócio de publicidade.
> *   **Avaliação de Clientes:** O Google Avaliações do Consumidor é um serviço que permite que lojistas coletem feedback de clientes que fizeram compras online. As avaliações podem ser exibidas como classificações nos anúncios do Google Search e no Google Shopping.


--------------------------------------------------------------

--- 📝 Resultado da Avaliação de Risco de Crédito ---

Nota de Risco: 2
Justificativa: Apesar do processo judicial e falta de informações sobre o endividamento e capital social, a Google possui um faturamento muito alto e é líder de mercado, o que indica um baixo risco de crédito.
--------------------------------------------------------------

--- 📝 Relatório de Crédito ---



> 
>     ## Relatório de Análise de Crédito
> 
>     **Nome da Empresa:** Nome da Empresa Não Informado
> 
>     ### Avaliação de Risco de Crédito
>     -   **Nota de Risco:** 2 (1 - Risco Muito Baixo, 5 - Risco Muito Alto)
>     -   **Justificativa:** Apesar do processo judicial e falta de informações sobre o endividamento e capital social, a Google possui um faturamento muito alto e é líder de mercado, o que indica um baixo risco de crédito.
> 
>     ### Decisão sobre Aumento de Crédito
>     Com base na análise das informações disponíveis, a recomendação é:
>     **Conceder o aumento de crédito.** A empresa apresenta baixo risco e boa capacidade de pagamento.
> 
>     ### Informações Detalhadas da Empresa:
>     -   **Com_base_nas_informações_disponíveis,_aqui_está_um_resumo_sobre_a_empresa_google:** 
> -   ***___**situação_cadastral:** ** Não foi encontrada informação direta sobre a situação cadastral da Google.
> -   ***___**data_de_abertura:** ** A Google foi fundada como empresa privada em 4 de setembro de 1998. O nome de domínio "Google" foi registrado em 15 de setembro de 1997.
> -   ***___**endereço:** ** A sede da Google é no Googleplex, em Mountain View, Califórnia, Estados Unidos. A Google possui escritórios em cerca de 60 países.
> -   ***___**telefone:** ** Não foi encontrado um número de telefone geral para contato com a Google.
> -   ***___**e-mail:** ** Não foi encontrado um e-mail geral para contato com a Google.
> -   ***___**atividade_econômica_principal_e_secundárias:** ** A Google atua nas áreas de internet, computação em nuvem, software, hardware, inteligência artificial e publicidade. A empresa lucra principalmente através da publicidade pelo AdWords.
> -   ***___**capital_social:** ** Essa informação não foi encontrada.
> -   ***___**quadro_de_sócios_e_administradores_(qsa):** ** Os fundadores são Larry Page e Sergey Brin. O CEO atual do Google e da Alphabet é Sundar Pichai. Anat Ashkenazi é a CFO e Thomas Kurian é o CEO da Google Cloud.
> -   ***___**faturamento_dos_últimos_3_anos:** **
> -   ***___2021:** A Alphabet (dona do Google) teve uma receita de US$ 257,637 bilhões e um lucro líquido de US$ 76,033 bilhões.
> -   ***___2022:** A receita do Google no terceiro trimestre foi de US$ 69,1 bilhões.
> -   ***___2020:** A empresa faturou 125,6 bilhões de dólares nos três primeiros trimestres.
> -   ***___**endividamento:** ** Não foram encontradas informações sobre o endividamento da Google.
> -   ***___**processos_judiciais_e_dívidas:** ** Em maio de 2025, a justiça dos EUA determinou que o Google divida seu negócio de publicidade.
> -   ***___**notícias_recentes_e_relevantes_sobre_a_empresa:** **
> -   ***___maio_de_2025:** A justiça dos EUA determinou que o Google divida seu negócio de publicidade.
> -   ***___**avaliação_de_clientes:** ** O Google Avaliações do Consumidor é um serviço que permite que lojistas coletem feedback de clientes que fizeram compras online. As avaliações podem ser exibidas como classificações nos anúncios do Google Search e no Google Shopping.


--------------------------------------------------------------

--- 📊 Gráfico de Risco de Crédito ---



alt.LayerChart(...)

--------------------------------------------------------------
